## Meals Count Backend Utilities Test
  
This notebook contains Python functionality that is incorporated as the MealsCount backend utility function **ParseXL** (see: [**[1]**](backend_utils.ipynb)). The code fragments below are tested on the CALPADs sample FRPM data in the form of an Excel file. 

In [84]:
import sys
import os
import pandas as pd
import numpy as np

In [85]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

METADATA_KEYS = ['Academic Year','View','As Of','Gender','School Type','School','User ID',
                 'Created Date','LEA']

METADATA_KEY_COLS = [0,2,4]
METADATA_VAL_COLS = [1,3,5]

DATA_L1_HDR_DICT = {'Non-Charter School(s)':'non-charter','Charter School(s)':'charter'}
DATA_L2_HDR_DICT = {'School Code':'school_code','School Name':'school_name','Total Enrollment':'total_enrolled',
                    'Free & Reduced Meal Program: 181/182':'frpm','Foster':'foster','Homeless (1)':'homeless',
                    'Migrant Program: 135':'migrant','Direct Certification':'direct_cert',
                    'Unduplicated Eligible Free/Reduced Meal Counts':'frpm_nodup',
                    'EL Funding Eligible (2)':'el', 'Total Unduplicated FRPM/EL Eligible (3)':'frpm_el_nodup'}

DATA_SUM_DICT = {'TOTAL - Selected Schools':'total','TOTAL LEA':'total_lea'}

DATA_L1_HDR_KEYS = ['Non-Charter School(s)','Charter School(s)']
DATA_L2_HDR_KEYS = ['School Code','School Name','Total Enrollment','Free & Reduced Meal Program: 181/182',
                    'Foster','Homeless(1)','Migrant Program: 135','Direct Certification',
                    'Unduplicated Eligible Free/Reduced Meal Counts','EL Funding Eligible (2)',
                    'Total Unduplicated FRPM/EL Eligible (3)']

DATA_SUM1_KEYS = ['TOTAL - Selected Schools']
DATA_SUM2_KEYS = ['TOTAL LEA']

INVALID_SCHOOL_CODE = 9999999
ALL_SCHOOL_TYPE = 'all'

In [86]:
xlfile = os.path.join(DATADIR,DATAFILE)

# intentionally not skipping rows here since we need the metadata too
xl = pd.ExcelFile(xlfile)                
tmpdf = xl.parse(xl.sheet_names[0])       

In [87]:
# intentionlly set to display the entire data-frame
pd.set_option('display.max_columns', 100)
tmpdf

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.17 - FRPM/English Learner/Foster Youth - Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Academic Year:,NaN,NaN,NaN,NaN,2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gender:,NaN,NaN,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,User ID:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,View:,NaN,NaN,NaN,NaN,ODS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,School Type:,NaN,NaN,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Created Date:,NaN,NaN,NaN,03-12-2018,NaN,NaN,NaN,NaN
4,NaN,NaN,As Of:,NaN,NaN,NaN,NaN,2018-03-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,School:,NaN,NaN,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LEA:,NaN,NaN,NaN,SomeCity Union High,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Non-Charter School(s),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free/Reduced Meal Eligibility Counts Based On:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,School Code,NaN,School Name,NaN,NaN,NaN,NaN,NaN,NaN,Total Enrollment,NaN,NaN,Free & Reduced Meal Program: 181/182,NaN,Foster,NaN,Homeless (1),NaN,NaN,Migrant Program: 135,NaN,NaN,Direct Certification,NaN,NaN,Unduplicated Eligible Free/Reduced Meal Counts,NaN,NaN,NaN,EL Funding Eligible (2),NaN,NaN,Total Unduplicated FRPM/EL Eligible (3),NaN,NaN,NaN,NaN
9,NaN,NaN,1000001,NaN,School NC01,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,NaN,4,NaN,27,NaN,0,NaN,NaN,0,NaN,NaN,6,NaN,NaN,29,NaN,NaN,NaN,5,NaN,NaN,30,NaN,NaN,NaN,NaN


In [88]:
# UNUSED
# count nulls in each row. high-value rows will contain meta-data and 
# column headers
#tmpdf_null_cnt = tmpdf.isnull().astype(int).sum(axis=1)
#tmpdf_null_cnt

In [89]:
# get the indices for the rows where the L1 headers are present
data_l1 = tmpdf.index[tmpdf[tmpdf.isin(DATA_L1_HDR_KEYS)].notnull().any(axis=1)].tolist()
data_l1

[6, 41]

In [90]:
# get indices for rows where the L2 headers are present
# these will indicate the beginning of data 
data_l2_begin = tmpdf.index[tmpdf[tmpdf.isin(DATA_L2_HDR_KEYS)].notnull().any(axis=1)].tolist()
data_l2_begin

[8, 43]

In [91]:
# get the indices for the rows where the misc headers are present
# these will indicate the end of data
data_l2_end = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM1_KEYS)].notnull().any(axis=1)].tolist()
data_l2_end

[39, 47]

In [92]:
# get any other keys that is part of data
data_other = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM2_KEYS)].notnull().any(axis=1)].tolist()
data_other

[48]

In [93]:
# generate indices of non-data rows 
metadata_idx = list(range(0,data_l1[0]))
n = len(DATA_L1_HDR_KEYS)

# TODO: malformed files may have any of the keys missing resulting in 
# empty lists of indices

for i in range(0,n):    
    metadata_idx += list(range(data_l1[i]+1,data_l2_begin[i]))
    if i < n-1:
        metadata_idx += list(range(data_l2_end[i]+1,data_l1[i+1]))

metadata_idx += list(range(data_l2_end[n-1]+1,data_other[0]))
metadata_idx += list(range(data_other[-1]+1,tmpdf.shape[0]))
metadata_idx        

[0, 1, 2, 3, 4, 5, 7, 40, 42, 49, 50, 51, 52, 53]

In [94]:
# copy metadata rows to its own dataframe
tmpdf_md = tmpdf.loc[metadata_idx,:]
# clean-up
tmpdf_md.dropna(axis=1,how='all',inplace=True)
tmpdf_md.dropna(axis=0,how='all',inplace=True)

# purge the same from the main dataframe
tmpdf.drop(metadata_idx,inplace=True)
# clean-up
tmpdf.dropna(axis=1,how='all',inplace=True)
tmpdf.dropna(axis=0,how='all',inplace=True)

In [95]:
# collect l1 header names
df_l1 = tmpdf.loc[data_l1]
df_l1 = df_l1.loc[:,df_l1.notnull().any()]
l1_hdrs = df_l1.T.unstack().tolist()
l1_hdrs = [s for s in l1_hdrs if str(s) != 'nan']
# drop all l1 headers
tmpdf.drop(data_l1,inplace=True)

In [96]:
def get_dataframe(row_idx,col_idx,school_type):
    
    data = tmpdf.loc[row_idx,:].values
    cols = tmpdf.loc[col_idx].values
    df = pd.DataFrame(data=data,columns=cols)        
    
    df['school_type'] = school_type 
    df['index'] = row_idx
    
    df.dropna(axis=1,how='all',inplace=True)
    df.dropna(axis=0,how='all',inplace=True)
    
    return df

# create a new ddtaframe for each school type
df_list = []
for i in range(0,n):
    
    row_idx = list(range(data_l2_begin[i]+1,data_l2_end[i]+1))   
    col_idx = data_l2_begin[i]
    school_type = l1_hdrs[i]    
    
    df_list.append(get_dataframe(row_idx,col_idx,school_type))
    
    if (i==n-1):
        row_idx = data_other
        df_list.append(get_dataframe(row_idx,col_idx,np.nan))
        
df_full = pd.concat(df_list,axis=0,ignore_index=True)

In [97]:
# recode column names
df_full.rename(columns=DATA_L2_HDR_DICT,inplace=True)

In [98]:
# recode school_type
df_full['school_type'] = df_full['school_type'].map(DATA_L1_HDR_DICT)

In [99]:
# recode other fields
cond = df_full['index'].isin(data_l2_end + data_other)
df_full.loc[cond,'school_name'] = df_full[cond]['school_code'].map(DATA_SUM_DICT)
df_full.loc[cond,'school_code'] = INVALID_SCHOOL_CODE
cond = df_full['index'].isin(data_other)
df_full.loc[cond,'school_type'] = ALL_SCHOOL_TYPE

In [100]:
df_full.drop(['index'],axis=1,inplace=True)

In [101]:
df_full = df_full[list(DATA_L2_HDR_DICT.values()) + ['school_type']]

In [102]:
df_full

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
0,1000001,School NC01,37,4,27,0,0,6,29,5,30,non-charter
1,1000002,School NC02,1111,503,2,7,0,215,527,122,556,non-charter
2,1000003,School NC03,2332,897,2,14,0,440,979,169,1037,non-charter
3,1000004,School NC03,854,733,6,10,1,361,765,296,792,non-charter
4,1000005,School NC04,1628,1119,7,17,0,640,1239,474,1333,non-charter
5,1000006,School NC05,856,666,4,21,0,359,703,295,746,non-charter
6,1000007,School NC06,2377,1581,4,46,1,939,1726,644,1892,non-charter
7,1000008,School NC07,77,50,1,0,0,16,54,19,55,non-charter
8,1000009,School NC08,3031,585,1,7,0,368,676,178,782,non-charter
9,1000010,School NC09,1708,395,1,4,0,228,431,112,478,non-charter


### Metadata  
  
Below code fragments extract metadata information from the Excel file and make it available as a Python dictionary.    

In [103]:
tmpdf_md

,Unnamed: 1,Unnamed: 2,Unnamed: 5,Unnamed: 7,Unnamed: 10,Unnamed: 12,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 19,Unnamed: 28,Unnamed: 30,Unnamed: 32,Unnamed: 34
0,NaN,NaN,NaN,NaN,1.17 - FRPM/English Learner/Foster Youth - Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Academic Year:,NaN,2017-2018,NaN,NaN,NaN,Gender:,NaN,ALL,NaN,User ID:,NaN,NaN
3,NaN,View:,NaN,ODS,NaN,NaN,NaN,School Type:,NaN,ALL,NaN,Created Date:,NaN,03-12-2018
4,NaN,As Of:,NaN,2018-03-12 00:00:00,NaN,NaN,NaN,School:,NaN,ALL,NaN,LEA:,NaN,SomeCity Union High
7,NaN,NaN,NaN,NaN,NaN,NaN,Free/Reduced Meal Eligibility Counts Based On:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,Free/Reduced Meal Eligibility Counts Based On:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,This report includes students with Primary and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Grade:,NaN,"01-First Grade,02-Second Grade,03-Third Grade,...",NaN,NaN,Ethnicity/Race:,NaN,NaN,ALL,NaN,5 - 17 Year Olds:,NaN,LCFF,NaN


In [104]:
# add appropriate prefix and suffix to metadata keys 
md_keys = ['   ' + s + ':' for s in METADATA_KEYS]
# get indices for rows where the metadata keywords are present
md_idx = tmpdf_md.index[tmpdf_md[tmpdf_md.isin(md_keys)].notnull().any(axis=1)].tolist()
md_idx

[2, 3, 4]

In [105]:
tmpdf_md = tmpdf_md.loc[md_idx,:]
tmpdf_md.dropna(axis=1,how='all',inplace=True)
tmpdf_md.dropna(axis=0,how='all',inplace=True)
tmpdf_md.columns = list(range(0,tmpdf_md.shape[1]))
tmpdf_md

,0,1,2,3,4,5
2,Academic Year:,2017-2018,Gender:,ALL,User ID:,NaN
3,View:,ODS,School Type:,ALL,Created Date:,03-12-2018
4,As Of:,2018-03-12 00:00:00,School:,ALL,LEA:,SomeCity Union High


In [110]:
%pprint

Pretty printing has been turned OFF


In [111]:
keys = list(tmpdf_md.loc[:,METADATA_KEY_COLS].unstack().values)
keys = list(map(str.strip,keys))
keys = list(map(str.lower,keys))
keys = [s.replace(' ','_') for s in keys]
keys = [s[:-1] for s in keys]
keys

['academic_year', 'view', 'as_of', 'gender', 'school_type', 'school', 'user_id', 'created_date', 'lea']

In [112]:
vals = list(tmpdf_md.loc[:,METADATA_VAL_COLS].unstack().values)
vals = [s.lower() if isinstance(s, str) else s for s in vals]
vals

['2017-2018', 'ods', datetime.datetime(2018, 3, 12, 0, 0), 'all', 'all', 'all', nan, '03-12-2018', 'somecity union high']

In [109]:
md_dict = dict(zip(keys, vals))
print(md_dict)

{'academic_year': '2017-2018', 'view': 'ods', 'as_of': datetime.datetime(2018, 3, 12, 0, 0), 'gender': 'all', 'school_type': 'all', 'school': 'all', 'user_id': nan, 'created_date': '03-12-2018', 'lea': 'somecity union high'}


### TODO  

* Test for error conditions